In [289]:
import pandas as pd
import random
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import Callback
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

In [2]:
columns = ["id", "event", "device", "channel", "code", "size", "data"]

def read_brain_data(file_path):
    # Read the file into a DataFrame
    df = pd.read_csv(file_path, sep='\t', names=columns, header=None)
    
    # Convert 'data' field from comma-separated string to list of floats/integers
    def process_data(data_str):
        # Detect integer vs float based on device type
        if df.iloc[0]['device'] in ["MW", "MU"]:
            return list(map(int, data_str.split(',')))
        else:
            return list(map(float, data_str.split(',')))
    
    # Apply processing to 'data' column
    df['data'] = df['data'].apply(process_data)
    
    return df

In [3]:
file_path = 'data/IN.txt'
data_df = read_brain_data(file_path)

In [4]:
data_df

,id,event,device,channel,code,size,data
0,1142043,173652,IN,AF3,0,256,"[4259.487179, 4237.948717, 4247.179487, 4242.0..."
1,1142044,173652,IN,AF4,0,256,"[4103.076923, 4100.51282, 4102.564102, 4087.69..."
2,1142045,173652,IN,T7,0,256,"[4245.128205, 4218.461538, 4242.051282, 4245.1..."
3,1142046,173652,IN,T8,0,256,"[4208.717948, 4188.717948, 4204.102564, 4198.4..."
4,1142047,173652,IN,PZ,0,256,"[4189.230769, 4203.589743, 4188.717948, 4186.6..."
...,...,...,...,...,...,...,...
65245,1207288,186701,IN,AF3,0,257,"[4214.358974, 4208.205128, 4214.871794, 4204.1..."
65246,1207289,186701,IN,AF4,0,257,"[4221.538461, 4211.794871, 4218.461538, 4213.3..."
65247,1207290,186701,IN,T7,0,257,"[4243.076923, 4233.846153, 4237.948717, 4219.4..."
65248,1207291,186701,IN,T8,0,257,"[4200.51282, 4191.282051, 4247.179487, 4246.66..."


In [96]:
class EEG_Dataset(Dataset):
    def __init__(self, data_df, sample_size=256):
        self.data = data_df.to_numpy()
        self.idxs_by_event = list(
            data_df.groupby("event")
            .apply(lambda x: x.index.tolist(), include_groups=False)
            .to_dict()
            .values()
        )
        self.nb_channel = len(self.idxs_by_event[0])
        self.sample_size = sample_size
        self.mean = np.mean(
            np.concatenate(
                [np.concatenate(self.data[idxs][:, -1]) for idxs in self.idxs_by_event]
            )
        )
        self.std = np.std(
            np.concatenate(
                [np.concatenate(self.data[idxs][:, -1]) for idxs in self.idxs_by_event]
            )
        )

    def __len__(self):
        return len(np.unique(self.data[:, 1]))

    def __getitem__(self, idx):
        data = np.vstack(self.data[self.idxs_by_event[idx]][:, -1])
        label = self.data[self.idxs_by_event[idx]][:, -3]
        normalized_data = (data - self.mean) / self.std
        return torch.tensor(normalized_data).view(-1, self.nb_channel).float(), label[0]

In [135]:
def collate(batches: list[torch.Tensor], sample_size, nb_channel):
    batch_size = len(batches)
    datas = [b[0] for b in batches]
    labels = torch.tensor([b[1] for b in batches]).reshape(-1)
    min_size = min(len(min(datas, key=len)), sample_size)
    datas = torch.vstack(
        [
            torch.cat([data[:min_size], torch.zeros(sample_size - min_size, nb_channel)])
            for data in datas
        ]
    ).view(batch_size, sample_size, -1)
    return datas.transpose(1,2), labels

In [345]:
class UNetAutoencoder(pl.LightningModule):
    def __init__(self, input_channels=5, sequence_length=256, latent_dim=64):
        super(UNetAutoencoder, self).__init__()

        # Encoder
        self.enc1 = self.conv_block(input_channels, 32)
        self.enc2 = self.conv_block(32, 64)
        self.enc3 = self.conv_block(64, 128)
        self.enc4 = self.conv_block(128, 256)

        # Calculate the size of the encoder output
        self.encoder_output_size = sequence_length // 16 * 256

        # Latent space
        self.fc1 = nn.Linear(self.encoder_output_size, latent_dim)
        self.fc2 = nn.Linear(latent_dim, self.encoder_output_size)

        # Decoder
        self.dec4 = self.conv_block(256, 128, transpose=True)
        self.dec3 = self.conv_block(256, 64, transpose=True)  # 256 due to skip connection
        self.dec2 = self.conv_block(128, 32, transpose=True)  # 128 due to skip connection
        self.dec1 = nn.ConvTranspose1d(64, input_channels, kernel_size=3, stride=2, padding=1, output_padding=1)

        self.criterion = nn.MSELoss()

    def conv_block(self, in_channels, out_channels, transpose=False):
        if not transpose:
            return nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=2, padding=1),
                nn.BatchNorm1d(out_channels),
                nn.ReLU(),
                nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1),
                nn.BatchNorm1d(out_channels),
                nn.ReLU()
            )
        else:
            return nn.Sequential(
                nn.ConvTranspose1d(in_channels, out_channels, kernel_size=3, stride=2, padding=1, output_padding=1),
                nn.BatchNorm1d(out_channels),
                nn.ReLU(),
                nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1),
                nn.BatchNorm1d(out_channels),
                nn.ReLU()
            )

    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)
        e2 = self.enc2(e1)
        e3 = self.enc3(e2)
        e4 = self.enc4(e3)

        # Flatten
        x = e4.view(e4.size(0), -1)

        # Latent space
        x = self.fc1(x)
        x = self.fc2(x)

        # Reshape for decoder
        x = x.view(x.size(0), 256, -1)

        # Decoder with skip connections
        x = self.dec4(x)
        x = torch.cat([x, e3], dim=1)
        x = self.dec3(x)
        x = torch.cat([x, e2], dim=1)
        x = self.dec2(x)
        x = torch.cat([x, e1], dim=1)
        x = self.dec1(x)

        return x

    def get_encoder(self):
        return nn.Sequential(
            self.enc1, self.enc2, self.enc3, self.enc4,
            nn.Flatten(),
            self.fc1
        )

    def training_step(self, batch, batch_idx):
        x, _ = batch
        x_hat = self.forward(x)
        loss = self.criterion(x_hat, x)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, _ = batch
        x_hat = self.forward(x)
        loss = self.criterion(x_hat, x)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",
            },
        }

In [158]:
class MNISTClassifier(pl.LightningModule):
    def __init__(self, latent_dim=64, num_classes=10):
        super(MNISTClassifier, self).__init__()
        
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, latent_dim)
        )
        
        self.classifier = nn.Linear(latent_dim, num_classes)
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        features = self.feature_extractor(x)
        return self.classifier(features), features

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits, _ = self(x)
        loss = self.criterion(logits, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits, _ = self(x)
        loss = self.criterion(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log("val_loss", loss)
        self.log("val_acc", acc)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

In [283]:
class EncodedDataset:
    def __init__(self, dataloader, encoder):
        super().__init__()
        self.encoded_datas, self.labels = self._encode_dataset(dataloader, encoder)

    def _encode_dataset(self, dataloader, encoder):
        encoded_datas = []
        labels = []
        for batch in dataloader:
            encoded_data = encoder(batch[0]).detach()
            encoded_datas.append(encoded_data)
            labels.append(batch[1])
        encoded_datas = torch.cat(encoded_datas)
        labels = torch.cat(labels)
        return encoded_datas, labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.encoded_datas[idx], self.labels[idx].item()


In [292]:
class LatentDataset(torch.utils.data.Dataset):
    def __init__(self, eeg_data, mnist_data):
        self.paired_data = self._pair_data(eeg_data, mnist_data)

    def _pair_data(self, eeg_data, mnist_data):
        eeg_by_label = {i: [] for i in range(10)}
        mnist_by_label = {i: [] for i in range(10)}

        # Group EEG data by label
        for eeg, label in eeg_data: 
            if 0 <= label < 10:
                eeg_by_label[label].append(eeg)

        # Group MNIST data by label
        for img, label in mnist_data:
            mnist_by_label[label].append(img)

        # Pair EEG and MNIST data
        paired_data = []
        for label in eeg_by_label.keys():
            eeg_samples = eeg_by_label[label]
            mnist_samples = mnist_by_label[label]

            # Use the maximum number of samples available
            n_samples = max(len(eeg_samples), len(mnist_samples))

            # Replicate samples if necessary
            if len(eeg_samples) < n_samples:
                eeg_samples = self._replicate_samples(eeg_samples, n_samples)
            if len(mnist_samples) < n_samples:
                mnist_samples = self._replicate_samples(mnist_samples, n_samples)

            for eeg, mnist in zip(eeg_samples, mnist_samples):
                paired_data.append((eeg, mnist, label))

        return paired_data

    def _replicate_samples(self, samples, target_size):
        """Replicate samples randomly to reach the target size."""
        while len(samples) < target_size:
            samples.append(random.choice(samples))
        return samples

    def __len__(self):
        return len(self.paired_data)

    def __getitem__(self, idx):
        return self.paired_data[idx]

In [341]:
class LatentProjection(pl.LightningModule):
    def __init__(self, eeg_latent_dim, mnist_latent_dim, hidden_dims=[256, 128]):
        super(LatentProjection, self).__init__()
        
        layers = []
        input_dim = eeg_latent_dim
        
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(input_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(0.2)
            ])
            input_dim = hidden_dim
        
        layers.append(nn.Linear(input_dim, mnist_latent_dim))
        
        self.projection = nn.Sequential(*layers)
        self.criterion = nn.MSELoss()

    def forward(self, x):
        return self.projection(x)

    def training_step(self, batch, batch_idx):
        eeg_latent, mnist_latent, _ = batch
        projected = self(eeg_latent)
        loss = self.criterion(projected, mnist_latent)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        eeg_latent, mnist_latent, _ = batch
        projected = self(eeg_latent)
        val_loss = self.criterion(projected, mnist_latent)
        self.log("val_loss", val_loss)
        return val_loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",
            },
        }

In [161]:
class LossTracker(Callback):
    def __init__(self):
        self.train_losses = []
        self.val_losses = []

    def on_train_epoch_end(self, trainer, pl_module):
        self.train_losses.append(trainer.callback_metrics["train_loss"].item())

    def on_validation_epoch_end(self, trainer, pl_module):
        self.val_losses.append(trainer.callback_metrics["val_loss"].item())

In [162]:
def plot_losses(train_losses, val_losses):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss over Epochs')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Print final losses
    print(f"Final training loss: {train_losses[-1]}")
    print(f"Final validation loss: {val_losses[-1]}")

## EEG

In [323]:
full_eeg_dataset = EEG_Dataset(data_df)

# Calculate sizes for train and validation sets
total_size = len(full_eeg_dataset)
train_size = int(0.8 * total_size)  # 80% for training
val_size = total_size - train_size  # Remaining 20% for validation

# Use random_split to create train and validation datasets
train_eeg_dataset, val_eeg_dataset = random_split(full_eeg_dataset, [train_size, val_size], 
                                          generator=torch.Generator().manual_seed(42))

In [325]:
batch_size = 32
eeg_train_dataloader = DataLoader(
    dataset=train_eeg_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=lambda batches: collate(
        batches, full_eeg_dataset.sample_size, full_eeg_dataset.nb_channel
    ),

)

eeg_val_dataloader = DataLoader(
    dataset=val_eeg_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=lambda batches: collate(
        batches, full_eeg_dataset.sample_size, full_eeg_dataset.nb_channel
    ),
)

In [326]:
latent_dim = 64

In [346]:
# checkpoint_path = "lightning_logs/version_16/checkpoints/epoch=9-step=5220.ckpt"
# autoencoder = CNNAutoencoder.load_from_checkpoint(checkpoint_path)
loss_tracker = LossTracker()
autoencoder = UNetAutoencoder(latent_dim=latent_dim)
trainer = Trainer(max_epochs=10, callbacks=[loss_tracker])
trainer.fit(autoencoder, eeg_train_dataloader, eeg_val_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/camilziane/My-projects/EEG/eeg-mnist/.venv/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "

   | Name      | Type            | Params
-----------------------------------------------
0  | enc1      | Sequential      | 3.7 K 
1  | enc2      | Sequential      | 18.8 K
2  | enc3      | Sequential      | 74.5 K
3  | enc4      | Sequential      | 296 K 
4  | fc1       | Linear          | 262 K 
5  | fc2       | Linear          | 266 K 
6  | dec4      | Sequential      | 148 K 
7  | dec3      | Sequential      | 61.8 K
8  | dec2      | Sequential      | 15.6 K
9  | dec1      | ConvTranspose1d | 965   
10 | criterion | MSELoss         | 0     
----

/Users/camilziane/My-projects/EEG/eeg-mnist/.venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/camilziane/My-projects/EEG/eeg-mnist/.venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 9: 100%|██████████| 327/327 [00:49<00:00,  6.61it/s, v_num=30, train_loss_step=0.0195, val_loss=0.0743, train_loss_epoch=0.0245] 

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 327/327 [00:49<00:00,  6.60it/s, v_num=30, train_loss_step=0.0195, val_loss=0.0743, train_loss_epoch=0.0245]


## MNIST

In [328]:
transform = transforms.Compose([transforms.ToTensor()])
mnist_full = datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
mnist_train_dataset, mnist_val_dataset = random_split(mnist_full, [55000, 5000])
mnist_train_loader = DataLoader(mnist_train_dataset, batch_size=64, shuffle=True)
mnist_val_loader = DataLoader(mnist_val_dataset, batch_size=64)

In [329]:
checkpoint_path = "lightning_logs/version_14/checkpoints/epoch=9-step=8600.ckpt"
mnist_classifier = MNISTClassifier.load_from_checkpoint(checkpoint_path)
# mnist_classifier = MNISTClassifier(latent_dim=latent_dim)
# mnist_trainer = pl.Trainer(max_epochs=3)
# mnist_trainer.fit(mnist_classifier, mnist_train_loader, mnist_val_loader)

## Model Alignement

In [348]:
eeg_encoder = autoencoder.get_encoder()
mnist_encoder = mnist_classifier.feature_extractor

In [349]:
eeg_train_encoded_dataset = EncodedDataset(eeg_train_dataloader, eeg_encoder)
eeg_val_encoded_dataset = EncodedDataset(eeg_val_dataloader, eeg_encoder)

In [332]:
mnist_train_encoded_data = EncodedDataset(mnist_train_loader, mnist_encoder)
mnist_val_encoded_data = EncodedDataset(mnist_val_loader, mnist_encoder)

In [333]:
latent_train_dataset = LatentDataset(
    eeg_train_encoded_dataset, mnist_train_encoded_data
)
latent_train_dataloader = DataLoader(latent_train_dataset, batch_size=32, shuffle=True)

latent_val_dataset = LatentDataset(eeg_val_encoded_dataset, mnist_train_encoded_data)
latent_val_dataloader = DataLoader(latent_val_dataset, batch_size=32, shuffle=False)

In [342]:
loss_tracker = LossTracker()
latent_projection = LatentProjection(latent_dim,latent_dim)
latent_trainer = Trainer(max_epochs=10)
latent_trainer.fit(latent_projection, latent_train_dataloader, latent_val_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs



  | Name       | Type       | Params
------------------------------------------
0 | projection | Sequential | 58.6 K
1 | criterion  | MSELoss    | 0     
------------------------------------------
58.6 K    Trainable params
0         Non-trainable params
58.6 K    Total params
0.234     Total estimated model params size (MB)


Epoch 7:  38%|███▊      | 648/1719 [00:06<00:11, 95.01it/s, v_num=28]       